# Sequential SAR Change Detection

In [1]:
from auxil.eeSar_seq import run
run()

TWFwKGJhc2VtYXA9eyd1cmwnOiAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLCAnbWF4X3pvb20nOiAxOSwgJ2F0dHJpYnV0aW9uJzogJ01hcCDigKY=


VkJveChjaGlsZHJlbj0oVGV4dGFyZWEodmFsdWU9dSdBbGdvcml0aG0gb3V0cHV0JywgZGlzYWJsZWQ9VHJ1ZSwgcm93cz0zKSwgSEJveChjaGlsZHJlbj0oVGV4dCh2YWx1ZT11JzIwMTctMDTigKY=
